In [ ]:
import math

base_time, base_fee, time_unit, fee_unit = 0, 0, 0, 0
total_fee = dict()
total_time = dict()
parked = dict()

def calculate_parking_time(car_id, e_h, e_m):
    global total_time, parked

    previous_parking_time = 0
    if car_id in total_time:
        previous_parking_time = total_time[car_id]
    # Calculate current parking time
    s_h, s_m = parked[car_id]
    diff_h, diff_m = e_h - s_h, e_m - s_m
    current_parking_time = diff_h * 60 + diff_m
    total_time[car_id] = previous_parking_time + current_parking_time

def calculate_total_fee():
    global total_time, total_fee
    global base_time, base_fee, time_unit, fee_unit

    for car_id, time in total_time.items():
        # 누적 주차 시간이 기본 시간이하라면, 기본 요금을 청구합
        if time < base_time:
            total_fee[car_id] = base_fee
        # 누적 주차 시간이 기본 시간을 초과하면,
        else:
            # 기본 요금에 더해서,
            fee = base_fee
            # 초과한 시간에 대해서 단위 시간 마다 단위 요금을 청구
            time = time - base_time
            # 초과한 시간이 단위 시간으로 나누어 떨어지지 않으면, 올림합니다.
            fee += (math.ceil(time / time_unit)) * fee_unit
            total_fee[car_id] = fee


def solution(fees, records):
    answer = []
    global total_time, parked, total_fee
    global base_time, base_fee, time_unit, fee_unit

    base_time, base_fee, time_unit, fee_unit = tuple(fees)

    for record in records:
        # records의 각 원소는 "시각 차량번호 내역"
        time, car_id, status = tuple(record.split())
        h, m = tuple(map(int, time.split(":")))
        if status == "IN":
            parked[car_id] = (h, m)
        elif status == "OUT":
            calculate_parking_time(car_id, h, m)
            parked.pop(car_id)
    # 입차된 후에 출차된 내역이 없다면, 23:59에 출차된 것으로 간주합니다.
    # method 1] get ket, calculate parking time, remove itme
    while parked:
        # get one of keys without popping item
        car_id = next(iter(parked))
        calculate_parking_time(car_id, 23, 59)
        parked.pop(car_id)
    # method 2] iterate over keys, calcualte parking time
    # for car_id, (s_h, s_m) in parked.items():
    #     calculate_parking_time(car_id, 23, 59)

    calculate_total_fee()
    # 차량 번호가 작은 자동차부터 청구할 주차 요금을 차례대로 정수 배열에 담아서 return
    total_ids = sorted(total_fee)
    for car_id in total_ids:
        answer.append(total_fee[car_id])

    return answer